In [1]:
#!pip install datasets
#!pip install boto3
#!pip install datasets
#!pip install -U bitsandbytes
#!pip install evaluate

In [1]:
#########################################################
##                PYTHON PACKAGE IMPORTS               ##
#########################################################
from transformers import AutoTokenizer
from peft import PeftModel
import pandas as pd
import boto3
import io

from google.colab import userdata
aws_access_key_id = userdata.get('aws_access_key_id')
aws_secret_access_key = userdata.get('aws_secret_access_key')

#!cp /content/drive/MyDrive/LoRaSequenceClassifier.py /content/
#!cp /content/drive/MyDrive/FinancialSentimentDataLoader.py /content/
#!cp /content/drive/MyDrive/SentimentInference.py /content/
#!cp /content/LoRaCausalLM.py /content/drive/MyDrive/

from FinancialSentimentDataLoader import FinancialSentimentDataLoader
from LoRaSequenceClassifier import LoRaSequenceClassifier
from SentimentInference import SentimentInference

aws_access_key_id = aws_access_key_id
aws_secret_access_key = aws_secret_access_key
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

#from huggingface_hub import notebook_login

In [3]:
#notebook_login()

In [2]:
############################################################
#####   LOAD TRAIN/TEST DATA AS HUGGINGFACE DATASETS   #####
############################################################
data_loader = FinancialSentimentDataLoader(aws_access_key_id, aws_secret_access_key)
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
training_data, test_data = data_loader.get_training_and_test_data(tokenizer)

Map:   0%|          | 0/41533 [00:00<?, ? examples/s]

Map:   0%|          | 0/9900 [00:00<?, ? examples/s]

In [3]:
###########################################################################
##  FINE-TUNE DEEPSEEK R1-DISTILLED VARIABLE MODEL LoRA CLASSIFICATION   ##
###########################################################################
classifier = LoRaSequenceClassifier(model_name=model_name)
classifier.fine_tune(train_dataset=training_data, eval_dataset=test_data, num_train_epochs=2, batch_size=1,)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Trainable parameters:
trainable params: 4,362,752 || all params: 1,548,081,664 || trainable%: 0.2818


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
500,0.752600,0.574063,0.690673,0.514763,0.539466,0.492222
1000,0.523400,0.452865,0.750842,0.630148,0.623664,0.636768
1500,0.404800,0.282216,0.878754,0.818799,0.815803,0.821818
2000,0.229100,0.197555,0.927340,0.891021,0.890931,0.891111
2500,0.199600,0.182806,0.940135,0.910383,0.908551,0.912222
3000,0.188600,0.168025,0.946263,0.919443,0.918886,0.920000
3500,0.187000,0.159024,0.951414,0.927184,0.926389,0.927980
4000,0.180000,0.149275,0.953401,0.930186,0.929061,0.931313
4500,0.163700,0.150777,0.955253,0.932936,0.932137,0.933737
5000,0.160900,0.145107,0.956397,0.934619,0.934289,0.934949


In [2]:
############################################################
##             GET FINANCIAL PHRASE BANK DATA             ##
############################################################
def get_financial_phrase_test_data():
    s3 = boto3.client(
        's3',
        aws_access_key_id=aws_access_key_id,
        aws_secret_access_key=aws_secret_access_key,
        region_name="us-east-1"
    )
    bucket_name = "harvard-capstone-bronze-bucket"
    file_key = "FinancialPhraseBank/Sentences_50Agree.csv"
    obj = s3.get_object(Bucket=bucket_name, Key=file_key)
    df = pd.read_csv(io.BytesIO(obj['Body'].read()))
    return df

df = get_financial_phrase_test_data()
df.head()

,title,sentiment
0,"According to Gran , the company has no plans t...",2
1,Technopolis plans to develop in stages an area...,2
2,The international electronic industry company ...,1
3,With the new production plant the company woul...,0
4,According to the company 's updated strategy f...,0


In [19]:
############################################################
##            INFERENCE ON FINBERT DATA BENCHMARK         ##
############################################################
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel
# Load inference model and tokenizer
label2id = {"positive": 0, "negative": 1, "neutral": 2},
id2label = {0: "positive", 1: "negative", 2: "neutral"}
tokenizer = AutoTokenizer.from_pretrained("./finetuned_classification_model")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})


inference_model = AutoModelForSequenceClassification.from_pretrained(
            "./finetuned_classification_model",
            num_labels=3,
            label2id=label2id,
            id2label=id2label,
            pad_token_id=tokenizer.pad_token_id
        )

#inference_model = PeftModel.from_pretrained(base, "./finetuned_classification_model")
#inference_model = inference_model.merge_and_unload()
#inference_model, tokenizer = classifier.get_inference_model()



Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
# Initialize SentimentInference class
sentiment_inference = SentimentInference(inference_model, tokenizer)

# Process the DataFrame and compute metrics
dataframe, accuracy, f1 = sentiment_inference.process_dataframe(df)

In [26]:
print("Processed DataFrame:")
print(dataframe.head())
print(f"\nAccuracy: {accuracy}")
print(f"F1 Score: {f1}")

Processed DataFrame:
                                               title  sentiment  prediction
0  According to Gran , the company has no plans t...          2           2
1  Technopolis plans to develop in stages an area...          2           2
2  The international electronic industry company ...          1           1
3  With the new production plant the company woul...          0           2
4  According to the company 's updated strategy f...          0           2

Accuracy: 0.7839455220800661
F1 Score: 0.7611420665190605
